In [3]:
from train_semi_auto import BracketFlowModule
from parenthesis import BracketDataset
from sampling import semiauto_euler_sampling


In [5]:
checkpoint_path = "checkpoints/bracket-flow/last-v48.ckpt"
model = BracketFlowModule.load_from_checkpoint(checkpoint_path)

In [10]:
steps = 200
batch_size = 10
samples, trace = semiauto_euler_sampling(model, model.interpolant, steps=steps, mask=0, pad=3, batch_size=10, max_length=64)

/n/netscratch/albergo_lab/Lab/brianlck/interpretable-flow/model/transformer.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/n/netscratch/albergo_lab/Lab/brianlck/interpretable-flow/model/transformer.py:167: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


In [11]:
from IPython.display import display, HTML
import json

# Select the index of the tensor to visualize
id = 0
assert 0 <= id < batch_size, "id must be in [0, batch_size)"


dt = 1. / steps
last = ""

e = []

for i in range(len(trace)):
    t = dt * i
    s = BracketDataset.parse_tensor(trace[i][id])
    

    for j, (a, b) in enumerate(zip(last, s)):
        if a != b:
            e.append(dict(
                t=t,
                a="c",
                tk=b,
                i=j
            ))
    
    
    if len(s) > len(last):
        for j in range(len(last), len(s)):
            e.append(dict(
                t=t,
                a="i",
                tk=s[j],
                i=j
            ))

    last = s


display(HTML(f'''
<style>
.token{{display:inline-block;padding:5px 10px;margin:2px;border:1px solid #aaa;border-radius:4px;
font-family:monospace;transition:background .5s}}
.hi-i{{background:lightgreen}}
.hi-c{{background:yellow}}
#s{{min-height:30px;margin-bottom:20px}}
</style>
<div id="s"></div>
<button id="r">Replay</button>
<script>
// Use events provided from Python
var ev = {json.dumps(e)}, s = [];
var render = function(hIdx, act){{
  var c = document.getElementById("s"); c.innerHTML="";
  s.forEach(function(t,k){{ 
    var sp = document.createElement("span");
    sp.className = "token" + (k===hIdx ? " hi-"+act : "");
    sp.textContent = t; c.appendChild(sp);
  }}); 
}},
run = function(){{
  s = []; render();
  ev.forEach(function(e){{
    setTimeout(function(){{
      if(e.a=="i"){{
        var p = e.i;
        s.splice(p, 0, e.tk);
        render(p, "i");
      }} else {{
        var p = (e.i===undefined || e.i<0 || e.i>=s.length) ? Math.floor(s.length/2) : e.i;
        s[p] = e.tk; render(p, "c");
      }}
      setTimeout(render,600);
    }}, e.t*5000);
  }});
}};
document.getElementById("r").onclick = run;
run();
</script>
'''))